<p>
<font size='8px'> <center> Preprocessing </center> </font>
<br>

<font>Both user and Tappy data given in form .txt files are processed converted to .xlsx. Columns with categorical values are converted to one hot vectors and cleaned (Nan for the rows with miss matching values for particluar columns or in case of Unknown Format)
</font>
<br><br>

<font>Main focus of this preprocessing by the end is to set dataset in such a way its easier to find time's correlation with parkisons
</font>
<br><br>

<font size='3px' > There are 3 main sections: </font>
<ol>
    <li> All User details .txt file are processed and saved in one file called User_details.xlsx 
        <br> original size: 41.6 KB
        <br>( 21.3 KB, 4 sec) </li>
    <br>
    <li> All tappy .txt files are processed and saved in form of .xlsx file 
        <br> Original size: 524 MB 
        <br>( 641 MB, ~2 hours ExcelWriter is slow on big datasets and also I was using jupyter python hence performace was affected ) 
    </li>
    <br>
    <li> Previously processed Tappy files are sorted based on timeline (date,time in asc order) and grouped later divided into sets based on date to be saved as &lt;User_unquieId&gt;_YYMMDD.xlsx this is done to make it easily feedable into RNN using a batch generator<br>
    ( MB , ~ 2 hour)
    </li>
</ol>

<font>
    Files are processed and saved in this manner so they are easily accessible and loadable at the time of training and Sorting based on date and time helps in directly feeding it into RNN or anyother Squence model
    </font>
    <br>
    <br>
    <font color='red'>Error</font> in dataset: &nbsp; " ZI1KGKLCD5_1612.txt " is empty 
</p>


In [ ]:
import pandas as pd
import numpy as np
import os

<font size='6px'> Part - I </font>

In [ ]:
# path to user dataset
user_dataset = "dataset/users"

# user attribute
user_list = ['user','BirthYear', 'Gender', 'Parkinsons', 'Tremors','DiagnosisYear', 
          'Sided', 'UPDRS', 'Impact', 'Levadopa', 'DA', 'MAOB', 'Other']

# Setting attribute list for user dataframe
user_dataframe = pd.DataFrame(columns  = user_list)

# list of all the names of files in user directory
user_file_list = os.listdir(user_dataset)

# Adding value to user to dataset
for i in range(len(user_file_list)):

    # fetch filename 
    filename = user_file_list[i]

    # name of user file is User_XXXXXXXXXX.txt
    # we are only interest in 10 alphanumeric unique id
    # convert '_' to '.' to ease the split function
    # finally getting the id at index: 1
    user_id = filename.replace('_','.').split('.')[1]
    
    # read the data
    _data = pd.read_table(user_dataset+'/'+filename,header=None,sep=':')
    
    # convert second column to a list (Please cross check the format)
    _data = _data[1].tolist()
    
    # Strip all the whitespace
    for ind in range(len(_data)):
        _data[ind] = _data[ind].replace(' ','')

    # Insert it into user_dataframe with unique User Id as first column value
    _data = [user_id] + _data

    user_dataframe.loc[i] = _data

<font size='4px'> Following shows the user attributes being converted to particular format </font>
<ul>
    <li> BirthYear : int </li><br>
    <li> Gender : <br> 
         &nbsp; female -> 0 and male -> 1 </li><br>
    <li> parkinsons :<br> 
         &nbsp; True -> 1 and False -> 0 </li><br>
    <li> Tremors :<br> 
         &nbsp; True -> 1 and False -> 0 </li><br>
    <li> DiagnosisYear : int ( if not given then Nan ) </li><br>
    <li> Sided: Divided into 3 columns ( L_Sided, None_Sided, R_sided ) </li><br> 
    <li> UPDRS is scale having value from 0 - 4 convert it to one-hot encoding vector when not known all values are zero
        <br> 0 = Absent.
        <br> 1 = Slight and infrequently present.
        <br> 2 = Mild in amplitude and persistent. Or moderate in amplitude, but only intermittently present.
        <br> 3 = Moderate in amplitude and present most of the time.
        <br> 4 = Marked in amplitude and present most of the time. 
    </li>
    <br>
    <li> Impact ( Mild/Medium/Severe ) to one-hot encoding vector  when not known all values are zero </li><br>
    <li> Levadopa:<br> 
         &nbsp; True -> 1 and False -> 0 </li><br>
    <li> DA:<br> 
         &nbsp; True -> 1 and False -> 0 </li><br>
    <li> MAOB:<br> 
         &nbsp; True -> 1 and False -> 0 </li><br>
    <li> Other:<br> 
         &nbsp; True -> 1 and False -> 0 </li>
 </ul>
 

In [ ]:
# ['user','BirthYear', 'Gender', 'Parkinsons', 'Tremors','DiagnosisYear', 'Sided', 'UPDRS',
#  'Impact', 'Levadopa', 'DA', 'MAOB', 'Other']

# converting  BirthYear and  DiagnosisYear to numeric
user_dataframe['BirthYear'] = pd.to_numeric(user_dataframe['BirthYear'],downcast="unsigned", errors ="coerce")
user_dataframe['DiagnosisYear'] = pd.to_numeric(user_dataframe['DiagnosisYear'],downcast="unsigned", errors ="coerce")

# Gender: String replacing it with int
user_dataframe['Gender'] = user_dataframe['Gender'].replace({"Female":0,"Male":1})

# Replace True with 1 and False with 0 keeping int dtype
for i in ['Parkinsons', 'Tremors','Levadopa', 'DA', 'MAOB', 'Other']:
    user_dataframe[i] = user_dataframe[i].replace({"True":1,"False":0})

# Mapping cleans the cloumns making any other avlue present to Nan
# Split the column into all the k columns (having k categories) 
# having drop_first = False gives us k columns hot encoding and not k-1
user_dataframe['Impact'] = user_dataframe['Impact'].map({'Medium':'Medium','Mild':'Mild','Severe':'Severe'})
user_dataframe = pd.get_dummies(user_dataframe, columns=['Impact'], drop_first=False)

# Creates two columns Sided_R and Sided_L
user_dataframe = pd.get_dummies(user_dataframe, columns=['Sided'], drop_first=False)

# 'Dont know' is changed to Nan
user_dataframe['UPDRS'] = user_dataframe['UPDRS'].map({'0':0,'1':1,'2':2,'3':3,'4':4})
user_dataframe = pd.get_dummies(user_dataframe, columns=['UPDRS'], drop_first=False)


# Save it for later use
if os.path.exists('processed_dataset/User_details.xlsx'):
    os.remove('processed_dataset/User_details.xlsx')

writer = pd.ExcelWriter('processed_dataset/User_details.xlsx')
user_dataframe.to_excel(writer,'Sheet1')
writer.save()

<font size='6px'> Part - II </font>

In [ ]:
# path to tappy dataset
tappy_dataset = "dataset/TappyData/"

# tappy data attribute
tappy_header = ['user','date','time','Hand_Kpressed',
                'Hold_time','Direction','Latency_time','Flight_time']


In [ ]:
# List all the files in directory
tappy_file_list = os.listdir(tappy_dataset)

# iterator through tappy data
for i in tappy_file_list[:]:
    
    # Print Filename currently working on
    print(i)
    
    userId_Tappy_df = pd.DataFrame(columns=tappy_header)
    
    
    _data = pd.read_table(tappy_dataset+i,header=None,sep='\t',error_bad_lines=False,low_memory=False)
    _data = _data.drop(_data.columns[8],axis=1)
    
    
    userId_Tappy_df['user'] = _data[0]

    userId_Tappy_df['date'] = pd.to_datetime(_data[1],format="%y%m%d",errors='coerce')        
    # In Case we need Date in terms of 3 columns year,month,day (make sure to add columns to dataframe first)
    #
    # _data[1] = pd.to_datetime(_data[1],format="%y%m%d")
    # userId_Tappy_df['year'] = pd.DatetimeIndex(_data[1]).year
    # userId_Tappy_df['month'] = pd.DatetimeIndex(_data[1]).month
    # userId_Tappy_df['day'] = pd.DatetimeIndex(_data[1]).day


    userId_Tappy_df['time'] = pd.to_datetime(_data[2],errors='coerce').dt.time
    # In Case we need Time in terms of 3 columns hour,minute,second (make sure to add columns to dataframe first)
    #
    #userId_Tappy_df['hour'] = pd.to_datetime(_data[2]).dt.hour
    #userId_Tappy_df['minute'] = pd.to_datetime(_data[2]).dt.minute
    #userId_Tappy_df['second'] = pd.to_datetime(_data[2]).dt.second


    userId_Tappy_df['Hand_Kpressed'] = _data[3].map({'L':'L','R':'R','S':'S'})
    userId_Tappy_df = pd.get_dummies(userId_Tappy_df, columns=['Hand_Kpressed'], drop_first=False)
    #
    # This is needed becasuse some datasets are so small that they dont have all the direction cases hence
    # get_dummies is not sufficient to handle such event
    #
    for index in ['L','R','S']:
        if 'Hand_Kpressed_'+ index not in userId_Tappy_df:
            userId_Tappy_df['Hand_Kpressed_'+ index] = 0


    userId_Tappy_df['Hold_time'] = pd.to_numeric(_data[4],downcast='float',errors='coerce')


    # One Hot Vector of Direction
    userId_Tappy_df['Direction'] = _data[5].map({'LR':'LR','RR':'RR','SR':'SR',
                                                 'LL':'LL','RL':'RL','SL':'SL',
                                                 'LS':'LS','RS':'RS','SS':'SS'})
    userId_Tappy_df = pd.get_dummies(userId_Tappy_df, columns=['Direction'], drop_first=False)
    #
    # In case of All 9 columns are not formed  ['LR','LL','LS','RR','RL','RS','SR','SL','SS']
    # create the column that dont exists and column add zero to it
    #
    # This is needed becasuse some datasets are so small that they dont have all the direction cases hence
    # get_dummies is not sufficient to handle such event
    #
    for index in ['LR','LL','LS','RR','RL','RS','SR','SL','SS']:
        if 'Direction_'+ index not in userId_Tappy_df:
            userId_Tappy_df['Direction_'+ index] = 0

    userId_Tappy_df['Latency_time'] = pd.to_numeric(_data[6],downcast='float',errors='coerce')

    userId_Tappy_df['Flight_time'] = pd.to_numeric(_data[7],downcast='float',errors='coerce')



    name = i.split('.')[0]
    if os.path.exists( 'processed_dataset/tappy_data/'+ name + '.xlsx'):
        pass #os.remove( 'processed_dataset/tappy_data/'+ name + '.xlsx')
    else:
        writer = pd.ExcelWriter( 'processed_dataset/tappy_data/'+ name + '.xlsx')
        userId_Tappy_df.to_excel(writer,'Sheet1')
        writer.save()
    
print("\n Completed ! ")


<font size='6px'> Part - III </font>

In [ ]:
processed_tappy = "processed_dataset/tappy_data/"
furtherP_tappydata = "processed_dataset/furtherProcessed_tappy_data/"

tappy_list = os.listdir(processed_tappy)


# map all the files related to specific user 
user_tappy = {}
for i in range(len(tappy_list)):
    
    # name of file gives us XXXXXXXXXX_YYMM.xlsx
    filename = tappy_list[i]
    
    df_excel = pd.read_excel(processed_tappy+filename)
    
    # group and divided by date and later sort
    gs = df_excel.groupby('date')
    for indv in gs.groups:
        
        # get the file in format of  XXXXXXXXXX_YYMM
        nFilename = filename.split('.')[0]
        new_df = gs.get_group(indv)
        
        # sort by time just in case its not sorted 
        new_df.sort_values(by=['time'])
        
        # new_df.iloc[0]['date'] return timestamp in YYYY-MM-DD HH:MM:SS format
        # extract only day
        day = new_df.iloc[0]['date'].day
        nFilename = furtherP_tappydata + nFilename + str(day) + '.xlsx'
        
        if os.path.exists(nFilename):
            print('\n\n'+ nFilename + ' already exists\n\n')
        else:  
            # write the new dataframe to excel
            writer = pd.ExcelWriter(nFilename)
            new_df.to_excel(writer,'Sheet1')
            writer.save()
        
        print(nFilename + ' written successfully')
    
print('\n\n Completed !')

<font size='6px'> Part - IV </font> (Additional)
<br>
<font>
    TO Group all the users dataset into a single file
</font>

In [ ]:
processed_tappy = "processed_dataset/tappy_data/"
furtherP_tappydata = "processed_dataset/furtherProcessed_tappy_data/"

tappy_list = os.listdir(processed_tappy)


# map all the files related to specific user 
user_tappy = {}
for i in range(len(tappy_list)):
    
    # name of file gives us XXXXXXXXXX_YYMM.xlsx
    filename = tappy_list[i]
    
    # Extracting the user_id
    user_id = filename.replace('_','.').split('.')[0]
    
    # Adding all the files as list to user_id as key 
    if user_id in user_tappy.keys():
        user_tappy[user_id].append(filename)
    else:
        user_tappy.update({user_id:[filename]})
        
for i in user_tappy.keys():
    
    print(user_tappy[i])
    
    # write it out
    file = furtherP_tappydata+ i+".xlsx"
    
    if os.path.exists(file):
        continue#os.remove(file)
    
    
    
    # Start writing from
    startrow=0
    for j in user_tappy[i]:
        writer = pd.ExcelWriter(file)
        #excel_names = user_tappy[i]
        excel = pd.ExcelFile(processed_tappy+j)
        #excels = [pd.ExcelFile(processed_tappy+name) for name in excel_names]
        #frames = [x.parse(x.sheet_names[0], header=None,index_col=None) for x in excels]

        # delete the first row for all frames except the first
        # i.e. remove the header row -- assumes it's the first
        
        frames = excel.parse(excel.sheet_names[0], header=None,index_col=None)
        #frames[1:] = [df[1:] for df in frames[1:]]

        #print(frames)
        header = frames.iloc[0].tolist()[1:]

        # print(header)
        # concatenate them..
        #combined = pd.concat(frames)
        #combined = combined.drop(columns=[0])

        df = pd.DataFrame(columns=header)
        
        for ind,v in enumerate(header):
            df[v] = frames[ind+1].iloc[1:]

        
        if startrow != 0:
            df.to_excel(writer,'Sheet1',startrow=startrow,header=False,index= False )
        else:
            df.to_excel(writer,'Sheet1',startrow=startrow, index = False)
            
        startrow= startrow + len(df)
    
        writer.save()

<font size='6px'> Part - V </font>Error Checking
<br>
<font>
    In the dataset of 622 tappy files in that some data belong to group of 49 users whose user details is not given 
    ( I.e birthyear , or whether they have parkinsons or they dont ) And in User details of 277 user their are 10 
    users whose Tappy data is not given.
</font>

In [ ]:
# Error Checking
import os

tappy_dataset = "dataset/TappyData/"
user_dataset = "dataset/users/"

user_file_list = os.listdir(user_dataset)
user_file_list = [w.replace('.txt', '') for w in user_file_list]
user_file_list = [w.replace('User_', '') for w in user_file_list]


tappy_dataset_list = os.listdir(tappy_dataset)

tappy_dataset_list = [w.replace('.txt', '') for w in tappy_dataset_list]
tappy_dataset_list = [w.split('_')[0] for w in tappy_dataset_list]


c=1
for i in set(tappy_dataset_list):
    if i not in set(user_file_list):
        print(i,"   c:",c)
        c = c+1

c =1
print("\n\n\n")
for i in set(user_file_list):
    if i not in set(tappy_dataset_list):
        print(i,"   c:",c)
        c = c+1